# Optical Flow example provided by the DALI Team @NVIDIA, refactored to personal use case

In [2]:
from __future__ import print_function
from __future__ import division
import os.path
import numpy as np

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops

from matplotlib import pyplot as plt

We have one sequence, I placed the directory `simulaton_images` in another dir named `sequences`, so the SequenceReader operator can be used (it requires such nested structure, the `simulation_images` will represent one sequence).

In [3]:
batch_size = 1
data_dir="sequences"
sequence="sequences/simulation_images"
sequence_length = len(os.listdir(sequence))

# sanity check
print(os.listdir(data_dir))
print(os.listdir(sequence))
print(sequence_length)

print(sorted(os.listdir(sequence)))

['simulation_images']
['SDO_int291180.jpg', 'SDO_int291600.jpg', 'SDO_int291420.jpg', 'SDO_int290160.jpg', 'SDO_int291060.jpg', 'SDO_int291660.jpg', 'SDO_int291780.jpg', 'SDO_int291540.jpg', 'SDO_int290940.jpg', 'SDO_int290220.jpg', 'SDO_int290040.jpg', 'SDO_int291360.jpg', 'SDO_int291000.jpg', 'SDO_int291300.jpg', 'SDO_int290100.jpg', 'SDO_int290280.jpg', 'SDO_int290820.jpg', 'SDO_int290520.jpg', 'SDO_int290760.jpg', 'SDO_int290340.jpg', 'SDO_int291480.jpg', 'SDO_int290880.jpg', 'SDO_int291240.jpg', 'SDO_int290400.jpg', 'SDO_int291720.jpg', 'SDO_int290580.jpg', 'SDO_int290700.jpg', 'SDO_int291120.jpg', 'SDO_int290460.jpg', 'SDO_int290640.jpg']
30
['SDO_int290040.jpg', 'SDO_int290100.jpg', 'SDO_int290160.jpg', 'SDO_int290220.jpg', 'SDO_int290280.jpg', 'SDO_int290340.jpg', 'SDO_int290400.jpg', 'SDO_int290460.jpg', 'SDO_int290520.jpg', 'SDO_int290580.jpg', 'SDO_int290640.jpg', 'SDO_int290700.jpg', 'SDO_int290760.jpg', 'SDO_int290820.jpg', 'SDO_int290880.jpg', 'SDO_int290940.jpg', 'SDO_in

Functions used for Optical flow visualization.  
The code comes from [Tomrunia's GitHub](https://github.com/tomrunia/OpticalFlow_Visualization "OpticalFlow_Visualization")

## Using Dali
### Define the Pipeline.  

Read the data using Sequence reader (file_root needs to point to the toplevel `sequences` dir, not the `simulation_images` one!

Files will be read in alphabetical order.

In [4]:
class OFPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(OFPipeline, self).__init__(batch_size, num_threads, device_id, seed=16)

#         self.input = ops.VideoReader(device="gpu", filenames=video_filename, sequence_length=sequence_length)
        self.input = ops.SequenceReader(file_root = data_dir, sequence_length = sequence_length)
        self.of_op = ops.OpticalFlow(device="gpu", output_format=4)

    def define_graph(self):
        seq = self.input(name="Reader")
        of = self.of_op(seq.gpu()) # need to transfer the files to GPU
        return of

### Build and run DALI Pipeline.

In [5]:
pipe = OFPipeline(batch_size=batch_size, num_threads=1, device_id=0)
pipe.build()
pipe_out = pipe.run()
flow_vector = np.array(pipe_out[0].as_cpu()[0])
print(flow_vector.shape)

np.save("flow_vectors.npy", flow_vector)

(29, 66, 66, 2)


flow_vector is numpy array, the first dimension is the index of the "frame" (we got one less than the inputs,


### Visualize results